## Camera preprocessing for yellow lines

Initialises a camera object and sets the parameters. Runs on a seperate thread. Applys a mask to the video feed which only allows the yellow range of colour.

In [ ]:
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import threading
from traitlets.config.configurable import SingletonConfigurable
import ipywidgets.widgets as widgets
from IPython.display import display

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()  # Monitor the color_value variable
    
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        # Set the camera resolution to HD1080 (1920x1080)
        init_params.camera_resolution = sl.RESOLUTION.HD1080  # Higher resolution
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : " + repr(status) + ". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()

        # Flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width based on the selected resolution
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while self.thread_runnning_flag == True:
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())

    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()

def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

# Create a camera object
camera = Camera()
camera.start()  # Start capturing the data

# Create widgets for displaying the image
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

def func(change):
    # Scale is necessary for real-time data displaying
    scale = 0.1
    resized_image = cv2.resize(change['new'], None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

    # Convert the image to HSV (Hue, Saturation, Value) color space
    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
    
    # Define the range of yellow in HSV space
    lower_yellow = np.array([20, 100, 100])  # Lower bound of yellow
    upper_yellow = np.array([40, 255, 255])  # Upper bound of yellow
    
    # Create a mask for yellow color
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    
    # Apply the mask to the original image
    yellow_image = cv2.bitwise_and(resized_image, resized_image, mask=yellow_mask)

    # Display the yellow filtered image
    display_color.value = bgr8_to_jpeg(yellow_image)

    # Display the depth image as usual
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(camera.depth_image, alpha=0.03), cv2.COLORMAP_JET)
    resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_depth.value = bgr8_to_jpeg(resized_depth_colormap)

camera.observe(func, names=['color_value'])
